## Identify relevant sentences

In [224]:
import pandas as pd
import pickle
import nltk
from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
import string
from geopy.geocoders import Nominatim
import collections
import pycountry

In [225]:
with open("clean_trainig_data.pkl", 'rb') as picklefile:
   df = pickle.load(picklefile)

In [226]:
df = df.assign(doc_id=[0 + i for i in xrange(len(df))])

In [227]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,Tag,language,doc_id
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,Conflict and violence,en,0
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,Disasters,en,1
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,Disasters,en,2
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,Disasters,en,3
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,Disasters,en,4


In [228]:
df.shape

(492, 13)

## Sentences with numbers

In [229]:
def sentences_with_numbers(text):
    text = text.replace("\n", " ")
    sentences = sent_tokenize(text)
    sent_w_num = []
    for line in sentences:
        if bool(re.search(r'\d', line)) == True:
            sent_w_num.append(line)
    return sent_w_num

In [230]:
df["sent_w_num"] = df.decode_text.apply(sentences_with_numbers)

In [231]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,Tag,language,doc_id,sent_w_num
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,Conflict and violence,en,0,[ United Nations S/2015/302 Security Counci...
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,Disasters,en,1,[Flash flooding across Afghanistan and Pakista...
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,Disasters,en,2,[Coordination Saves Lives In the first week of...
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,Disasters,en,3,"[Afghanistan state news agency, Bakhtar News A..."
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,Disasters,en,4,[According to reports from Chinese state news ...


## Sentences with numbers AND people units or household units

In [232]:
people_units = ['people', 'peoples', 'persons', 'individuals', 'children', 'inhabitants',
                'residents', 'migrants', 'members']

household_units = ['families', 'households', 'houses', 'homes', 'community']

In [233]:
def lines_people(alist):
    idetect_people = []
    for word in people_units:
        for line in alist:
            if word in line: # set operations? set intersection?
                idetect_people.append(line)
    return idetect_people

In [234]:
def lines_households(alist):
    idetect_household = []
    for word in household_units:
        for line in alist:
            if word in line:
                idetect_household.append(line)
    return idetect_household

In [235]:
df['sent_w_num_p'] = df.sent_w_num.apply(lines_people)
df['sent_w_num_h'] = df.sent_w_num.apply(lines_households)

In [236]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,Tag,language,doc_id,sent_w_num,sent_w_num_p,sent_w_num_h
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,Conflict and violence,en,0,[ United Nations S/2015/302 Security Counci...,"[Approximately 15,000 people in northern areas...",[The attack resulted in the loss of three live...
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,Disasters,en,1,[Flash flooding across Afghanistan and Pakista...,"[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi..."
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,Disasters,en,2,[Coordination Saves Lives In the first week of...,[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., To d..."
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,Disasters,en,3,"[Afghanistan state news agency, Bakhtar News A...","[Afghanistan state news agency, Bakhtar News A...",[It was early Saturday when a flash flood hit ...
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,Disasters,en,4,[According to reports from Chinese state news ...,[],[Flash floods struck in the Shuhada district o...


## Sentences with displacement terms

In [237]:
displacement_terms = ['displace', 'displaced', 'evacuate', 'evacuated', 'forced', 'flee','fled','homeless', 'relief', 
         'sheltered', 'shelters', 'relocated', 'relocate', 'destroyed', 'washed', 'affected', 'destroyed', 
                'uninhabitable', 'submerged', 'killed', 'leave', 'placed', 'dead', 'died' ]

displace_people = ['displace', 'displaced', 'evacuate', 'evacuated', 'forced', 'flee','fled','homeless', 'relief', 
         'sheltered', 'shelters', 'relocated', 'relocate', 'destroyed', 'washed', 'affected', 'destroyed', 
                'uninhabitable', 'submerged', 'killed', 'leave', 'placed', 'dead', 'died' ]

displace_homes = ['displace', 'displaced', 'evacuate', 'evacuated', 'forced', 'flee','fled','homeless', 'relief', 
         'sheltered', 'shelters', 'relocated', 'relocate', 'destroyed', 'washed', 'affected', 'destroyed', 
                'uninhabitable', 'submerged', 'killed', 'leave', 'placed', 'dead', 'died' ]

In [238]:
def lines_w_terms(alist):
    idetect_term = []
    for word in displacement_terms:
        for line in alist:
            if word in line:
                idetect_term.append(line)
    return list(set(idetect_term))

In [239]:
df['sent_num_p_t'] = df.sent_w_num_p.apply(lines_w_terms)
df['sent_num_h_t'] = df.sent_w_num_h.apply(lines_w_terms)

In [240]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,Tag,language,doc_id,sent_w_num,sent_w_num_p,sent_w_num_h,sent_num_p_t,sent_num_h_t
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,Conflict and violence,en,0,[ United Nations S/2015/302 Security Counci...,"[Approximately 15,000 people in northern areas...",[The attack resulted in the loss of three live...,"[An additional 6,800 internally displaced pers...","[Similarly, among the individuals captured and..."
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,Disasters,en,1,[Flash flooding across Afghanistan and Pakista...,"[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi..."
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,Disasters,en,2,[Coordination Saves Lives In the first week of...,[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., To d...",[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., Acco..."
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,Disasters,en,3,"[Afghanistan state news agency, Bakhtar News A...","[Afghanistan state news agency, Bakhtar News A...",[It was early Saturday when a flash flood hit ...,[BNA report that at least 7 people were killed...,[BNA report that at least 7 people were killed...
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,Disasters,en,4,[According to reports from Chinese state news ...,[],[Flash floods struck in the Shuhada district o...,[],[Quoting an official from the Badakhshan provi...


# Find the country

In [244]:
def word_tokenize_filter(sentence):
    words = word_tokenize(sentence)
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    filtered_words = [c for c in filtered_words if c not in string.punctuation]
    return filtered_words

In [245]:
df['doc_word_tokens'] = df.decode_text.apply(word_tokenize_filter)

In [246]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,Tag,language,doc_id,sent_w_num,sent_w_num_p,sent_w_num_h,sent_num_p_t,sent_num_h_t,doc_word_tokens
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,Conflict and violence,en,0,[ United Nations S/2015/302 Security Counci...,"[Approximately 15,000 people in northern areas...",[The attack resulted in the loss of three live...,"[An additional 6,800 internally displaced pers...","[Similarly, among the individuals captured and...","[United, Nations, S/2015/302, Security, Counci..."
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,Disasters,en,1,[Flash flooding across Afghanistan and Pakista...,"[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Flash, flooding, across, Afghanistan, Pakista..."
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,Disasters,en,2,[Coordination Saves Lives In the first week of...,[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., To d...",[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., Acco...","[www.unocha.org, The, mission, United, Nations..."
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,Disasters,en,3,"[Afghanistan state news agency, Bakhtar News A...","[Afghanistan state news agency, Bakhtar News A...",[It was early Saturday when a flash flood hit ...,[BNA report that at least 7 people were killed...,[BNA report that at least 7 people were killed...,"[Afghanistan, state, news, agency, Bakhtar, Ne..."
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,Disasters,en,4,[According to reports from Chinese state news ...,[],[Flash floods struck in the Shuhada district o...,[],[Quoting an official from the Badakhshan provi...,"[Flash, floods, struck, Badakhshan, province, ..."


In [247]:
def one_location2(locations):
    countries = []
    for location in locations:
        try:
            country = pycountry.countries.get(name=location)
            if country.numeric > 0:
                country = country.name
                countries.append(country)
        except:
            pass
    return countries

In [248]:
df['countries'] = df.doc_word_tokens.apply(one_location2)

In [264]:
def country(countries):
    try:
        counter = collections.Counter(countries)
        country = counter.most_common(1)[0][0]
        one_country = country.encode("ascii","ignore")
        return one_country
    except:
        pass

In [265]:
df['one_country'] = df.countries.apply(country)

In [268]:
df.head()

,authors,date,image,keywords,summary,text,url,valid,decode_text,error,...,language,doc_id,sent_w_num,sent_w_num_p,sent_w_num_h,sent_num_p_t,sent_num_h_t,doc_word_tokens,countries,one_country
0,NaN,NaN,NaN,NaN,NaN,United Nations S/2015/302 Security Council...,http://www.securitycouncilreport.org/atf/cf/%7...,NaN,United Nations S/2015/302 Security Council...,all good,...,en,0,[ United Nations S/2015/302 Security Counci...,"[Approximately 15,000 people in northern areas...",[The attack resulted in the loss of three live...,"[An additional 6,800 internally displaced pers...","[Similarly, among the individuals captured and...","[United, Nations, S/2015/302, Security, Counci...","[Sudan, Sudan, Sudan, Sudan, Sudan, Sudan, Sud...",Sudan
1,[Heather Saul],2013-08-05 12:33:51+01:00,https://static.independent.co.uk/s3fs-public/t...,"[pakistan, homes, remote, afghanistan, floodin...",Flash flooding across Afghanistan and Pakistan...,Flash flooding across Afghanistan and Pakistan...,http://www.independent.co.uk/news/world/asia/1...,True,Flash flooding across Afghanistan and Pakistan...,all good,...,en,1,[Flash flooding across Afghanistan and Pakista...,"[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Mountainous Afghanistan was the worst hit, wi...","[Flash, flooding, across, Afghanistan, Pakista...","[Afghanistan, Pakistan, Afghanistan, Pakistan]",Afghanistan
2,NaN,NaN,NaN,NaN,NaN,www.unocha.org The mission of the United Natio...,http://reliefweb.int/sites/reliefweb.int/files...,NaN,www.unocha.org The mission of the United Natio...,all good,...,en,2,[Coordination Saves Lives In the first week of...,[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., To d...",[Coordination Saves Lives In the first week of...,"[More than 1,200 families are displaced., Acco...","[www.unocha.org, The, mission, United, Nations...",[],None
3,[],2015-05-11 09:15:05+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[baghlan, afghanistan, agency, province, distr...",Flash floods struck on 08 May 2015 in Faryab P...,"Afghanistan state news agency, Bakhtar News Ag...",http://floodlist.com/asia/afghanistan-flash-fl...,True,"Afghanistan state news agency, Bakhtar News Ag...",all good,...,en,3,"[Afghanistan state news agency, Bakhtar News A...","[Afghanistan state news agency, Bakhtar News A...",[It was early Saturday when a flash flood hit ...,[BNA report that at least 7 people were killed...,[BNA report that at least 7 people were killed...,"[Afghanistan, state, news, agency, Bakhtar, Ne...",[Afghanistan],Afghanistan
4,[],2015-07-27 11:42:21+00:00,http://floodlist.com/wp-content/uploads/2015/0...,"[province, official, hit, district, floods, fl...",According to reports from Chinese state news a...,Flash floods have struck once again in the Bad...,http://floodlist.com/asia/afghanistan-6-dead-f...,True,Flash floods have struck once again in the Bad...,all good,...,en,4,[According to reports from Chinese state news ...,[],[Flash floods struck in the Shuhada district o...,[],[Quoting an official from the Badakhshan provi...,"[Flash, floods, struck, Badakhshan, province, ...","[Afghanistan, Afghanistan]",Afghanistan


In [267]:
with open("training_sentences_w_units_terms.pkl", "wb") as picklefile:
    pickle.dump(df, picklefile)